<a href="https://colab.research.google.com/github/iskra3138/UDA/blob/main/Office31_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Mar 20 19:59:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Downloading source and dataset

### source files

In [2]:
!git clone https://github.com/tim-learn/SHOT.git

Cloning into 'SHOT'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 227 (delta 11), reused 15 (delta 6), pack-reused 203
Receiving objects: 100% (227/227), 489.24 KiB | 21.27 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [3]:
cd SHOT/object/

/content/SHOT/object


### office31 dataset

In [4]:
!gdown  https://drive.google.com/uc?id=0B4IapRTv9pJ1WGZVd1VDMmhwdlE

Downloading...
From: https://drive.google.com/uc?id=0B4IapRTv9pJ1WGZVd1VDMmhwdlE
To: /content/SHOT/object/domain_adaptation_images.tar.gz
77.2MB [00:01, 66.2MB/s]


In [5]:
!tar -xvzf domain_adaptation_images.tar.gz

amazon/images/back_pack/frame_0001.jpg
amazon/images/back_pack/frame_0002.jpg
amazon/images/back_pack/frame_0003.jpg
amazon/images/back_pack/frame_0004.jpg
amazon/images/back_pack/frame_0005.jpg
amazon/images/back_pack/frame_0006.jpg
amazon/images/back_pack/frame_0007.jpg
amazon/images/back_pack/frame_0008.jpg
amazon/images/back_pack/frame_0009.jpg
amazon/images/back_pack/frame_0010.jpg
amazon/images/back_pack/frame_0011.jpg
amazon/images/back_pack/frame_0012.jpg
amazon/images/back_pack/frame_0013.jpg
amazon/images/back_pack/frame_0014.jpg
amazon/images/back_pack/frame_0015.jpg
amazon/images/back_pack/frame_0016.jpg
amazon/images/back_pack/frame_0017.jpg
amazon/images/back_pack/frame_0018.jpg
amazon/images/back_pack/frame_0019.jpg
amazon/images/back_pack/frame_0020.jpg
amazon/images/back_pack/frame_0021.jpg
amazon/images/back_pack/frame_0022.jpg
amazon/images/back_pack/frame_0023.jpg
amazon/images/back_pack/frame_0024.jpg
amazon/images/back_pack/frame_0025.jpg
amazon/images/back_pack/f

In [6]:
ll

total 75504
drwxr-xr-x 3 root     4096 Mar 20 19:59 amazon/
-rw-r--r-- 1 root     2797 Mar 20 19:59 data_list.py
-rw-r--r-- 1 root 77187046 Mar 20 19:59 domain_adaptation_images.tar.gz
drwxr-xr-x 3 root     4096 Mar 20 19:59 dslr/
-rw-r--r-- 1 root     9903 Mar 20 19:59 image_multisource.py
-rw-r--r-- 1 root    14024 Mar 20 19:59 image_multitarget.py
-rw-r--r-- 1 root    11369 Mar 20 19:59 image_pretrained.py
-rw-r--r-- 1 root    16232 Mar 20 19:59 image_source.py
-rw-r--r-- 1 root    16393 Mar 20 19:59 image_target_oda.py
-rw-r--r-- 1 root    15741 Mar 20 19:59 image_target.py
-rw-r--r-- 1 root     3361 Mar 20 19:59 loss.py
-rw-r--r-- 1 root     5193 Mar 20 19:59 network.py
-rw-r--r-- 1 root     3312 Mar 20 19:59 run.sh
drwxr-xr-x 3 root     4096 Mar 20 19:59 webcam/


# Modifying 

### Making files about paths of images
- ref: https://github.com/tim-learn/SHOT/issues/2

In [7]:
import os, glob 
data_path = './data/office'
if not os.path.exists(data_path) :
  os.makedirs(data_path)

datasets = ['amazon', 'dslr', 'webcam']
classes = os.listdir('./amazon/images/')

for dataset in datasets:
  img_paths_file = open(os.path.join(data_path,'{}_list.txt'.format(dataset)), 'w')
  for i, cls in enumerate(classes): 
    for filename in glob.glob(os.path.join(dataset,'images',cls,'*.jpg')):
      img_paths_file.write("{} {}\n".format(filename,i))
  img_paths_file.close()

In [ ]:
### you can download txt files to check them.
#from google.colab import files
#files.download('./data/office/amazon_list.txt')
#files.download('./data/office/dslr_list.txt')
#files.download('./data/office/webcam_list.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
NUM_WORKERS = 4
BATCHSIZE = 64
folder = './data/'
names = ['amazon', 'dslr', 'webcam']
CLASS_NUM = 31
SOURCE = names[1]
TARGET = names[0]
TRTE = 'val'
DSET = 'office'
NET = 'resnet50'
MAX_EPOCH=100

NAME_SRC = SOURCE[0].upper()

S_DSET_PATH = folder + DSET + '/' + '{}_list.txt'.format(SOURCE)
TEST_DSET_PATH = folder + DSET + '/' + '{}_list.txt'.format(TARGET)

In [9]:
import argparse
import os, sys
import os.path as osp
import torchvision
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import network, loss
from torch.utils.data import DataLoader
from data_list import ImageList
import random, pdb, math, copy
from tqdm import tqdm
from loss import CrossEntropyLabelSmooth
from scipy.spatial.distance import cdist
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans


def image_train(resize_size=256, crop_size=224, alexnet=False):
  if not alexnet:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  else:
    normalize = Normalize(meanfile='./ilsvrc_2012_mean.npy')
  return  transforms.Compose([
        transforms.Resize((resize_size, resize_size)),
        transforms.RandomCrop(crop_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

def image_test(resize_size=256, crop_size=224, alexnet=False):
  if not alexnet:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  else:
    normalize = Normalize(meanfile='./ilsvrc_2012_mean.npy')
  return  transforms.Compose([
        transforms.Resize((resize_size, resize_size)),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        normalize
    ])
  
def data_load(): 
    ## prepare data
    dsets = {}
    dset_loaders = {}
    train_bs = BATCHSIZE
    txt_src = open(S_DSET_PATH).readlines()
    txt_test = open(TEST_DSET_PATH).readlines()

    if TRTE == "val":
        dsize = len(txt_src)
        tr_size = int(0.9*dsize)
        # print(dsize, tr_size, dsize - tr_size)
        tr_txt, te_txt = torch.utils.data.random_split(txt_src, [tr_size, dsize - tr_size])
    else:
        dsize = len(txt_src)
        tr_size = int(0.9*dsize)
        _, te_txt = torch.utils.data.random_split(txt_src, [tr_size, dsize - tr_size])
        tr_txt = txt_src

    dsets["source_tr"] = ImageList(tr_txt, transform=image_train())
    dset_loaders["source_tr"] = DataLoader(dsets["source_tr"], batch_size=train_bs, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)
    dsets["source_te"] = ImageList(te_txt, transform=image_test())
    dset_loaders["source_te"] = DataLoader(dsets["source_te"], batch_size=train_bs, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)
    dsets["test"] = ImageList(txt_test, transform=image_test())
    dset_loaders["test"] = DataLoader(dsets["test"], batch_size=train_bs*2, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)

    return dset_loaders


In [10]:
def op_copy(optimizer):
    for param_group in optimizer.param_groups:
        param_group['lr0'] = param_group['lr']
    return optimizer

def lr_scheduler(optimizer, iter_num, max_iter, gamma=10, power=0.75):
    decay = (1 + gamma * iter_num / max_iter) ** (-power)
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr0'] * decay
        param_group['weight_decay'] = 1e-3
        param_group['momentum'] = 0.9
        param_group['nesterov'] = True
    return optimizer

In [11]:
dset_loaders = data_load()
## set base network
netF = network.ResBase(res_name=NET).cuda()

netB = network.feat_bootleneck(type="bn", feature_dim=netF.in_features, bottleneck_dim=256).cuda()
netC = network.feat_classifier(type="wn", class_num = 31, bottleneck_dim=256).cuda()

param_group = []
learning_rate = 0.01
for k, v in netF.named_parameters():
    param_group += [{'params': v, 'lr': learning_rate*0.1}]
for k, v in netB.named_parameters():
    param_group += [{'params': v, 'lr': learning_rate}]
for k, v in netC.named_parameters():
    param_group += [{'params': v, 'lr': learning_rate}]   
optimizer = optim.SGD(param_group)
optimizer = op_copy(optimizer)

acc_init = 0
max_iter = MAX_EPOCH * len(dset_loaders["source_tr"])
interval_iter = max_iter // 10
iter_num = 0

netF.train()
netB.train()
netC.train()


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


feat_classifier(
  (fc): Linear(in_features=256, out_features=31, bias=True)
)

In [12]:
def cal_acc(loader, netF, netB, netC, flag=False):
    start_test = True
    with torch.no_grad():
        iter_test = iter(loader)
        for i, _ in enumerate(loader):
        #for i in range(len(loader)):  
            data = iter_test.next()
            inputs = data[0]
            labels = data[1]
            inputs = inputs.cuda()
            outputs = netC(netB(netF(inputs)))
            if start_test:
                all_output = outputs.float().cpu()
                all_label = labels.float()
                start_test = False
            else:
                all_output = torch.cat((all_output, outputs.float().cpu()), 0)
                all_label = torch.cat((all_label, labels.float()), 0)

    all_output = nn.Softmax(dim=1)(all_output)
    _, predict = torch.max(all_output, 1)
    accuracy = torch.sum(torch.squeeze(predict).float() == all_label).item() / float(all_label.size()[0])
    mean_ent = torch.mean(loss.Entropy(all_output)).cpu().data.item()
   
    if flag:
        matrix = confusion_matrix(all_label, torch.squeeze(predict).float())
        acc = matrix.diagonal()/matrix.sum(axis=1) * 100
        aacc = acc.mean()
        aa = [str(np.round(i, 2)) for i in acc]
        acc = ' '.join(aa)
        return aacc, acc
    else:
        return accuracy*100, mean_ent

def cal_acc_oda(loader, netF, netB, netC):
    start_test = True
    with torch.no_grad():
        iter_test = iter(loader)
        for i in range(len(loader)):
            data = iter_test.next()
            inputs = data[0]
            labels = data[1]
            inputs = inputs.cuda()
            outputs = netC(netB(netF(inputs)))
            if start_test:
                all_output = outputs.float().cpu()
                all_label = labels.float()
                start_test = False
            else:
                all_output = torch.cat((all_output, outputs.float().cpu()), 0)
                all_label = torch.cat((all_label, labels.float()), 0)

    all_output = nn.Softmax(dim=1)(all_output)
    _, predict = torch.max(all_output, 1)
    ent = torch.sum(-all_output * torch.log(all_output + 1e-5), dim=1) / np.log(CLASS_NUM)
    ent = ent.float().cpu()
    initc = np.array([[0], [1]])
    kmeans = KMeans(n_clusters=2, random_state=0, init=initc, n_init=1).fit(ent.reshape(-1,1))
    threshold = (kmeans.cluster_centers_).mean()

    predict[ent>threshold] = CLASS_NUM
    matrix = confusion_matrix(all_label, torch.squeeze(predict).float())
    matrix = matrix[np.unique(all_label).astype(int),:]

    acc = matrix.diagonal()/matrix.sum(axis=1) * 100
    unknown_acc = acc[-1:].item()

    return np.mean(acc[:-1]), np.mean(acc), unknown_acc
    # return np.mean(acc), np.mean(acc[:-1])

In [ ]:
iter_num = 0
while iter_num < max_iter: 
    iter_source = iter(dset_loaders["source_tr"])
    inputs_source, labels_source = iter_source.next()

    if inputs_source.size(0) == 1:
        continue

    iter_num += 1
    lr_scheduler(optimizer, iter_num=iter_num, max_iter=max_iter)

    inputs_source, labels_source = inputs_source.cuda(), labels_source.cuda()
    outputs_source = netC(netB(netF(inputs_source)))
    classifier_loss = CrossEntropyLabelSmooth(num_classes=31, epsilon=0.1)(outputs_source, labels_source) ## 다른부분           
    
    optimizer.zero_grad()
    classifier_loss.backward()
    optimizer.step()

    if iter_num % interval_iter == 0 or iter_num == max_iter:
        netF.eval()
        netB.eval()
        netC.eval()
        if DSET=='VISDA-C':
            acc_s_te, acc_list = cal_acc(dset_loaders['source_te'], netF, netB, netC, True)
            log_str = 'Task: {}, Iter:{}/{}; Accuracy = {:.2f}%'.format(NAME_SRC, iter_num, max_iter, acc_s_te) + '\n' + acc_list
        else:
            acc_s_te, _ = cal_acc(dset_loaders['source_te'], netF, netB, netC, False)
            log_str = 'Task: {}, Iter:{}/{}; Accuracy = {:.2f}%'.format(NAME_SRC, iter_num, max_iter, acc_s_te)
        #args.out_file.write(log_str + '\n')
        #args.out_file.flush()
        print(log_str+'\n')

        if acc_s_te >= acc_init:
            acc_init = acc_s_te
            best_netF = netF.state_dict()
            best_netB = netB.state_dict()
            best_netC = netC.state_dict()

        netF.train()
        netB.train()
        netC.train()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
netF.eval()
netB.eval()
netC.eval()

acc_s_te, _ = cal_acc(dset_loaders['test'], netF, netB, netC, False)
log_str = 'Task: {}, Iter:{}/{}; Accuracy = {:.2f}%'.format(NAME_SRC, iter_num, max_iter, acc_s_te)

print(log_str+'\n')